In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../../data/data_raw/tweets.csv", encoding='latin-1', names=["target", "ids", "date", "flag", "user", "text"])
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
df.drop(["target", "ids", "date", "flag", "user"], axis=1, inplace=True)
df.to_csv("../../data/data_human/tweets.csv", index=False)

In [1]:
from vllm import LLM, SamplingParams

INFO 02-11 04:26:46 __init__.py:190] Automatically detected platform cuda.


In [2]:
llm = LLM(model="meta-llama/Llama-3.2-1B-Instruct", dtype="half", max_model_len = 10_000)

WARNING 02-11 04:26:50 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-11 04:27:04 config.py:542] This model supports multiple tasks: {'classify', 'reward', 'generate', 'embed', 'score'}. Defaulting to 'generate'.
INFO 02-11 04:27:04 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=10000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, c

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-11 04:27:49 model_runner.py:1115] Loading model weights took 2.3185 GB
INFO 02-11 04:27:51 worker.py:267] Memory profiling takes 1.84 seconds
INFO 02-11 04:27:51 worker.py:267] the current vLLM instance can use total_gpu_memory (6.00GiB) x gpu_memory_utilization (0.90) = 5.40GiB
INFO 02-11 04:27:51 worker.py:267] model weights take 2.32GiB; non_torch_memory takes 0.04GiB; PyTorch activation peak memory takes 1.21GiB; the rest of the memory reserved for KV Cache is 1.84GiB.
INFO 02-11 04:27:52 executor_base.py:110] # CUDA blocks: 3761, # CPU blocks: 8192
INFO 02-11 04:27:52 executor_base.py:115] Maximum concurrency for 10000 tokens per request: 6.02x
INFO 02-11 04:27:57 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utiliz

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:21<00:00,  1.62it/s]

INFO 02-11 04:28:19 model_runner.py:1562] Graph capturing finished in 19 secs, took 0.12 GiB
INFO 02-11 04:28:19 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 29.80 seconds


In [5]:
sampling_params = SamplingParams(
    temperature=1.0,
    top_p=0.95,
)

In [6]:
prompts = [
    "Write a short story about a space explorer.",
    "Explain the concept of quantum entanglement in simple terms.",
    "Generate a creative recipe using chocolate and strawberries."
]

In [14]:
prompts *= 2000
len(prompts)

120000

In [15]:
outputs = llm.generate(prompts, sampling_params)

Processed prompts:   2%|▏         | 2564/120000 [04:23<3:09:03, 10.35it/s, est. speed input: 110.38 toks/s, output: 155.82 toks/s]

KeyboardInterrupt: 

In [9]:
outputs

[RequestOutput(request_id=0, prompt='Write a short story about a space explorer.', prompt_token_ids=[128000, 8144, 264, 2875, 3446, 922, 264, 3634, 54067, 13], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' Captain Orion Blackwood led his crew, the Starsong, on a perilous', token_ids=(22022, 69773, 5348, 6798, 6197, 813, 13941, 11, 279, 25676, 647, 11, 389, 264, 60392, 788), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1739235408.3704019, last_token_time=1739235409.2165337, first_scheduled_time=1739235408.4217725, first_token_time=1739235408.619185, time_in_queue=0.05137062072753906, finished_time=1739235409.2183104, scheduler_time=0.01740836900034992, model_forward_time=None, model_execute_time=None), lora_request=None, num_cached_tokens=0, multi_modal_placeholders={}),
 RequestOutput(request_id=1, prompt='Explain the concept of q

In [13]:
result = [sample.outputs[0].text for sample in outputs]

In [3]:
import csv
from itertools import islice
from vllm import LLM, SamplingParams

def batchify(iterable, batch_size):
    """Splits an iterable into smaller batches."""
    iterable = iter(iterable)
    while batch := list(islice(iterable, batch_size)):
        yield batch

def save_to_csv(filename, prompts, responses):
    """Saves prompts and responses to a CSV file."""
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for prompt, response in zip(prompts, responses):
            writer.writerow([prompt, response])

def generate_responses(prompts, temperature=0.8, top_p=0.95, max_tokens=100):
    """
    Generate a batch of outputs using vLLM with customizable sampling parameters.

    Args:
        prompts (list of str): List of input prompts.
        temperature (float): Sampling temperature.
        top_p (float): Nucleus sampling parameter.
        max_tokens (int): Maximum number of tokens in the output.

    Returns:
        list of str: Generated outputs corresponding to each input prompt.
    """
    
    # Define sampling parameters
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens
    )
    
    # Generate responses
    outputs = llm.generate(prompts, sampling_params)
    
    # Extract text responses
    return [output.outputs[0].text for output in outputs]

if __name__ == "__main__":
    # Example prompts
    prompts = [
        "Write a short story about a space explorer.",
        "Explain the concept of quantum entanglement in simple terms.",
        "Generate a creative recipe using chocolate and strawberries."
    ] * 10
    
    batch_size = 2  # Adjust based on GPU memory
    csv_filename = "responses.csv"
    
    # Write CSV header
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Prompt", "Response"])
    
    # Process in batches and save results
    for batch in batchify(prompts, batch_size):
        responses = generate_responses(batch, temperature=0.7, top_p=0.9, max_tokens=150)
        save_to_csv(csv_filename, batch, responses)
    
    print(f"Responses saved to {csv_filename}")


Processed prompts: 100%|██████████| 2/2 [00:02<00:00,  1.06s/it, est. speed input: 11.31 toks/s, output: 141.34 toks/s]

Responses saved to responses.csv


In [4]:
import pandas as pd

In [5]:
tmp = pd.read_csv("responses.csv")

In [6]:
tmp

,Prompt,Response
0,Write a short story about a space explorer.,Captain Zara Blackwood was a renowned space e...
1,Explain the concept of quantum entanglement in...,"Imagine you have two toy boxes, one in New Yo..."
2,Generate a creative recipe using chocolate and...,Chocolate Strawberry Tartlets:\nIngredients:\...
3,Write a short story about a space explorer.,Captain Jameson stood on the bridge of his sh...
4,Explain the concept of quantum entanglement in...,Imagine you have two toy cars that are connec...
5,Generate a creative recipe using chocolate and...,Strawberry Chocolate Tart with Caramelized Pe...
6,Write a short story about a space explorer.,Captain Zara Blackwood gazed out at the stars...
7,Explain the concept of quantum entanglement in...,"Imagine you have two toy cars, one red and on..."
8,Generate a creative recipe using chocolate and...,Strawberry Shortcake with Chocolate Mousse an...
9,Write a short story about a space explorer.,Captain Orion Blackwood stood on the edge of ...


In [7]:
tmp["Response"][0]

' Captain Zara Blackwood was a renowned space explorer who had spent her entire career studying the mysteries of the cosmos. She had explored countless worlds, discovered new planets, and encountered alien species. But despite all her success, Zara felt a sense of restlessness, a feeling that there was still so much to explore, so many secrets to uncover.\n\nOne day, Zara received a cryptic message from an unknown sender, hinting at the existence of a long-lost planet on the edge of the galaxy. The message read: "Meet me at the old observatory on planet Xylophia-IV at midnight. Come alone."\n\nZara\'s curiosity was piqued, and she knew she had to investigate. She assembled her trust'